<a href="https://colab.research.google.com/github/sdll/cifar-100-densenet/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image classification demo of DenseNet in PyTorch on CIFAR 100
## Configuration

In [0]:
# Taken from https://git.io/JeD2H
CIFAR_100_MEAN = (129.3, 124.1, 112.4)
CIFAR_100_STD = (68.2,  65.4,  70.4)

PROJECT = "fastrino"
DEVICE = "cuda"

In [3]:
!pip install gsheet-keyring ipython-secrets comet_ml

     |████████████████████████████████| 163kB 9.5MB/s 
     |████████████████████████████████| 409kB 52.7MB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 2.3MB 44.3MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=4c05709daaeeaa7df64981471ad0210154811a035deab1cac80be14d1d9a54e6
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


## Model

In [0]:
from comet_ml import Experiment
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor


class DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(DenseLayer, self).__init__()
        self.add_module("norm1", nn.BatchNorm2d(num_input_features)),
        self.add_module("relu1", nn.ReLU(inplace=True)),
        self.add_module(
            "conv1",
            nn.Conv2d(
                num_input_features,
                bn_size * growth_rate,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        ),
        self.add_module("norm2", nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module("relu2", nn.ReLU(inplace=True)),
        self.add_module(
            "conv2",
            nn.Conv2d(
                bn_size * growth_rate,
                growth_rate,
                kernel_size=3,
                stride=1,
                padding=1,
                bias=False,
            ),
        ),
        self.drop_rate = float(drop_rate)

    def bottleneck_function(self, inputs):
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(
            self.relu1(self.norm1(concated_features))
        )
        return bottleneck_output

    def forward(self, input_tensor):
        if isinstance(input_tensor, Tensor):
            prev_features = [input_tensor]
        else:
            prev_features = input_tensor

        bottleneck_output = self.bottleneck_function(prev_features)

        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        if self.drop_rate > 0:
            new_features = F.dropout(
                new_features, p=self.drop_rate, training=self.training
            )
        return new_features


class DenseBlock(nn.ModuleDict):
    def __init__(
        self, num_layers, num_input_features, bn_size, growth_rate, drop_rate
    ):
        super(DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
            )
            self.add_module("dense_layer%d" % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)


class Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(Transition, self).__init__()
        self.add_module("norm", nn.BatchNorm2d(num_input_features))
        self.add_module("relu", nn.ReLU(inplace=True))
        self.add_module(
            "conv",
            nn.Conv2d(
                num_input_features,
                num_output_features,
                kernel_size=1,
                stride=1,
                bias=False,
            ),
        )
        self.add_module("pool", nn.AvgPool2d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks"
     <https://arxiv.org/pdf/1608.06993.pdf>`_

    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first
        convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    __constants__ = ["features"]

    def __init__(
        self,
        growth_rate=32,
        block_config=(6, 12, 24, 16),
        num_init_features=64,
        bn_size=4,
        drop_rate=0,
        num_classes=1000,
        memory_efficient=False,
    ):

        super(DenseNet, self).__init__()

        self.features = nn.Sequential(
            OrderedDict(
                [
                    (
                        "conv0",
                        nn.Conv2d(
                            3,
                            num_init_features,
                            kernel_size=7,
                            stride=2,
                            padding=3,
                            bias=False,
                        ),
                    ),
                    ("norm0", nn.BatchNorm2d(num_init_features)),
                    ("relu0", nn.ReLU(inplace=True)),
                    (
                        "pool0",
                        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                    ),
                ]
            )
        )

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
            )
            self.features.add_module("dense_block%d" % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = Transition(
                    num_input_features=num_features,
                    num_output_features=num_features // 2,
                )
                self.features.add_module("transition%d" % (i + 1), trans)
                num_features = num_features // 2

        self.features.add_module("norm5", nn.BatchNorm2d(num_features))

        self.classifier = nn.Linear(num_features, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out


## Utilities

In [0]:
import argparse
from getpass import getpass

from ipython_secrets import *
import numpy as np
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.autograd import Variable
from tqdm import tqdm

from sklearn.metrics import accuracy_score, f1_score

def compute_metric(metric, labels, predictions, **kwargs):
    return metric(
        labels.cpu().data.numpy(), 
        torch.argmax(predictions.cpu(), dim=1).data.numpy(),
        **kwargs
    )

def read_args():
    parser = argparse.ArgumentParser(
        description="Image classification demo "
        + "of DenseNet in PyTorch on CIFAR 100"
    )

    parser.add_argument(
        "--num-epochs",
        default=25,
        type=int,
        help="Number of total epochs to run",
    )

    parser.add_argument(
        "--batch-size", default=128, type=int, help="Batch size (default: 128)"
    )

    parser.add_argument(
        "--lr",
        "--learning-rate",
        default=1e-4,
        type=float,
        help="Initial learning rate (default: 1e-4)",
    )    

    parser.add_argument(
        "--dropout", default=0.1, type=float, help="Dropout rate (default: 0.1)"
    )

    parser.add_argument(
        "--comet-logging",
        default=True,
        help="Whether to log progress to TensorBoard",
        action="store_true",
    )
    return parser

In [6]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=np.asarray(CIFAR_100_MEAN) / 255.0,
            std=np.asarray(CIFAR_100_STD) / 255.0,
        ),
    ]
)
data_dirname = "./data"

train_data = datasets.CIFAR100(
    root=data_dirname, train=True, download=True, transform=transform
)
test_data = datasets.CIFAR100(
    root=data_dirname, train=False, download=True, transform=transform
)

args = read_args().parse_args(args=[])
args.growth_rate = 32
args.block_config = (6, 12, 24, 16)
args.num_init_features = 64
args.bn_size = 4
args.num_classes = 1000

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=args.batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
)

test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

if args.comet_logging:
    comet_ml_api_key = get_secret("comet-{}".format(PROJECT))

    experiment = Experiment(
        api_key=comet_ml_api_key,
        project_name=PROJECT,
        workspace=PROJECT,
        auto_output_logging=None,
    )
    experiment.log_parameters(vars(args))

model = DenseNet(
    growth_rate=args.growth_rate,
    block_config=args.block_config,
    num_init_features=args.num_init_features,
    bn_size=args.bn_size,
    drop_rate=args.dropout,
    num_classes=args.num_classes,
)
model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

with experiment.train():
    model.train()
    step = 0
    for epoch in range(1, args.num_epochs + 1):
        experiment.log_current_epoch(epoch)
        with tqdm(
            total=len(train_data) // args.batch_size,
            desc="Epoch {}: Step".format(epoch),
        ) as pbar:
            for idx, (images, labels) in enumerate(train_loader):
                images = Variable(images.to(DEVICE))
                labels = Variable(labels.to(DEVICE))

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                experiment.set_step(step)
                experiment.log_metric("loss", loss.data.item())

                predictions = torch.nn.functional.softmax(outputs, dim=0)
                accuracy = compute_metric(accuracy_score, labels, predictions)
                f1 = compute_metric(
                    f1_score, labels, predictions, average="macro"
                )

                experiment.log_metrics(
                    {"accuracy": accuracy, "f1": f1}, step=step
                )
                step += 1
                if (idx + 1) % 95 == 0:
                    print("\n")
                    print("-" * 80)
                    print(
                        "Epoch [%d/%d], Step [%d/%d], Loss: %.4f"
                        % (
                            epoch,
                            args.num_epochs,
                            idx + 1,
                            len(train_data) // args.batch_size,
                            loss.data.item(),
                        )
                        + ", Accuracy: {:.4f}".format(accuracy)
                        + ", F1: {:.4f}".format(f1)
                    )
                    print("-" * 80)
                pbar.update()

0it [00:00, ?it/s]

169009152it [00:06, 27750593.70it/s]                               


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/fastrino/fastrino/11700425158b4ee2a3c02a2fa3e335a1

Epoch 1: Step:   0%|          | 0/390 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
Epoch 1: Step:  24%|██▍       | 95/390 [00:25<01:17,  3.80it/s]



--------------------------------------------------------------------------------
Epoch [1/25], Step [95/390], Loss: 4.4298, Accuracy: 0.0625, F1: 0.0289
--------------------------------------------------------------------------------


...

Epoch 25: Step:  73%|███████▎  | 285/390 [01:16<00:27,  3.78it/s]



--------------------------------------------------------------------------------
Epoch [25/25], Step [285/390], Loss: 0.6321, Accuracy: 0.5938, F1: 0.5155
--------------------------------------------------------------------------------


Epoch 25: Step:  97%|█████████▋| 380/390 [01:42<00:02,  3.68it/s]



--------------------------------------------------------------------------------
Epoch [25/25], Step [380/390], Loss: 0.9144, Accuracy: 0.6016, F1: 0.5078
--------------------------------------------------------------------------------


Epoch 25: Step: 391it [01:44,  4.05it/s]                         


In [7]:
with experiment.test():
    model.eval()

    total_accuracy = 0
    total_f1 = 0
    count = 0
    with tqdm(
        total=len(test_data) // args.batch_size, desc="Test Step".format(epoch)
    ) as pbar:
        for idx, (images, labels) in enumerate(test_loader):
            images = Variable(images.to(DEVICE))
            labels = Variable(labels.to(DEVICE))

            outputs = model(images)
            loss = criterion(outputs, labels)

            predictions = torch.nn.functional.softmax(outputs, dim=0)
            count += 1
            total_accuracy += compute_metric(
                accuracy_score, labels, predictions
            )
            total_f1 += compute_metric(
                f1_score, labels, predictions, average="macro"
            )
            pbar.update()

    experiment.log_metrics(
        {"mean_accuracy": total_accuracy / count, "mean_f1": total_f1 / count},
        step=step,
    )
    print("\n")
    print("-" * 80)
    print(
        "Mean test batch accuracy on {} images with batch size {}: {}".format(
            len(test_data), args.batch_size, total_accuracy / count
        )
    )
    print(
        "Mean test batch F1 on {} images with batch size {}: {}".format(
            len(test_data), args.batch_size, total_f1 / count
        )
    )
    print("-" * 80)
    experiment.end()

Test Step:   0%|          | 0/78 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
Test Step: 79it [00:05, 15.21it/s]                        
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/fastrino/fastrino/11700425158b4ee2a3c02a2fa3e335a1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     sys.cpu.percent.01                  : (6.9, 6.9)
COMET INFO:     sys.cpu.percent.02                  : (13.5, 13.5)
COMET INFO:     sys.cpu.percent.avg                 : (10.2, 10.2)
COMET I



--------------------------------------------------------------------------------
Mean test batch accuracy on 10000 images with batch size 128: 0.33079509493670883
Mean test batch F1 on 10000 images with batch size 128: 0.2666806530771708
--------------------------------------------------------------------------------


COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
